In [ ]:
import xarray as xr
import cfgrib
%pylab inline
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import warnings
warnings.filterwarnings('ignore')
from wrf import smooth2d
import matplotlib.transforms as mtransforms

def get_z(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa','shortName': 'gh'}})
    var = file.get('gh')[12].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var


In [ ]:
min_lat = 27
max_lat = 55
min_lon = -105
max_lon = -65

z1_a = get_z('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111500.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
z1_f = get_z('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f036.grib2',
                 min_lat, max_lat, min_lon, max_lon)

z2_a = get_z('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111512.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
z2_f = get_z('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f048.grib2',
                 min_lat, max_lat, min_lon, max_lon)

z3_a = get_z('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111600.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
z3_f = get_z('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f060.grib2',
                 min_lat, max_lat, min_lon, max_lon)

z4_a = get_z('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111612.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
z4_f = get_z('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f072.grib2',
                 min_lat, max_lat, min_lon, max_lon)


In [ ]:
fig, axd = plt.subplot_mosaic([['topleft','topright','cbar'],['bottomleft','bottomright','cbar']],
                              constrained_layout=True, figsize=(8,5), dpi=2000, 
                              gridspec_kw={'width_ratios':[1,1,0.05],'height_ratios':[1,1]},
                              per_subplot_kw={'topleft':{'projection':crs.PlateCarree()},
                                              'topright':{'projection':crs.PlateCarree()},
                                              'bottomleft':{'projection':crs.PlateCarree()},
                                              'bottomright':{'projection':crs.PlateCarree()}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

axd['topleft'].set_extent((min_lon,max_lon,min_lat,max_lat-4))

cs1 = axd['topleft'].contour(z1_a['longitude'], z1_a['latitude'], smooth2d(z1_a['gh'],2)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='k')
cs2 = axd['topleft'].contour(z1_f['longitude'], z1_f['latitude'], smooth2d(z1_f['gh'],2)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['topleft'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='magenta')
im1 = axd['topleft'].contourf(z1_a['longitude'], z1_a['latitude'], z1_f['gh'] - z1_a['gh'],
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*15,extend='both',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=False
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-100, -90, -80, -70])
gls.ylocator = mticker.FixedLocator([30, 40, 50])

axd['topleft'].set_title('0000 UTC 15 Nov 2018 ($t$ = 36 h)')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
#############

axd['topright'].set_extent((min_lon,max_lon,min_lat,max_lat-4))

cs1 = axd['topright'].contour(z2_a['longitude'], z2_a['latitude'], smooth2d(z2_a['gh'],2)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='k')
cs2 = axd['topright'].contour(z2_f['longitude'], z2_f['latitude'], smooth2d(z2_f['gh'],2)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['topright'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='magenta')
im1 = axd['topright'].contourf(z2_a['longitude'], z2_a['latitude'], z2_f['gh'] - z2_a['gh'],
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*15,extend='both',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=False
gls.right_labels=False
gls.left_labels=False
gls.xlocator = mticker.FixedLocator([-100, -90, -80, -70])
gls.ylocator = mticker.FixedLocator([30, 40, 50])

axd['topright'].set_title('1200 UTC 15 Nov 2018 ($t$ = 48 h)')
axd['topright'].text(0.0, 1.0, '(b)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
#############

axd['bottomleft'].set_extent((min_lon,max_lon,min_lat,max_lat-4))

cs1 = axd['bottomleft'].contour(z3_a['longitude'], z3_a['latitude'], smooth2d(z3_a['gh'],2)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='k')
cs2 = axd['bottomleft'].contour(z3_f['longitude'], z3_f['latitude'], smooth2d(z3_f['gh'],2)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='magenta')
im1 = axd['bottomleft'].contourf(z3_a['longitude'], z3_a['latitude'], z3_f['gh'] - z3_a['gh'],
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*15,extend='both',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-100, -90, -80, -70])
gls.ylocator = mticker.FixedLocator([30, 40, 50])

axd['bottomleft'].set_title('0000 UTC 16 Nov 2018 ($t$ = 60 h)')
axd['bottomleft'].text(0.0, 1.0, '(c)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)


##############
### BOTTOM RIGHT
#############

axd['bottomright'].set_extent((min_lon,max_lon,min_lat,max_lat-4))

cs1 = axd['bottomright'].contour(z4_a['longitude'], z4_a['latitude'], smooth2d(z4_a['gh'],2)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='k')
cs2 = axd['bottomright'].contour(z4_f['longitude'], z4_f['latitude'], smooth2d(z4_f['gh'],2)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,20),fontsize=6,colors='magenta')
im1 = axd['bottomright'].contourf(z4_a['longitude'], z4_a['latitude'], z4_f['gh'] - z4_a['gh'],
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*15,extend='both',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=0.6, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=False
gls.xlocator = mticker.FixedLocator([-100, -90, -80, -70])
gls.ylocator = mticker.FixedLocator([30, 40, 50])

axd['bottomright'].set_title('1200 UTC 16 Nov 2018 ($t$ = 72 h)')
axd['bottomright'].text(0.0, 1.0, '(d)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='vertical', cax=axd['cbar'])

plt.savefig('/p/work1/lloveras/real/nov2018/figs_nov2018/gfs_cutoff.pdf',bbox_inches='tight')
plt.show()
